In [0]:
%run "/Repos/HealthCare project/HealthCare Project/Configuration/config file"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType
from pyspark.sql.functions import col

class History():

    def __init__(self):
        obj_conf = ConfigModule()
        self.raw_file_dir = f"{obj_conf.base_file_path}/raw"
        self.catalog = obj_conf.environment
        self.schema = obj_conf.db_name
    
    def history_load(self):
        self.json_schema = StructType([StructField ('date', DateType()),\
                                  StructField ('week', StringType()),
                                  StructField ('year', StringType()),
                                  StructField ('month', StringType()),
                                  StructField ('dayofweek', StringType()),
                                  StructField ('dayofmonth', StringType()),
                                  StructField ('dayofyear', StringType()),
                                  StructField ('week_part', StringType())
                                  ])
        df = spark.read.format('json').schema(self.json_schema).load(self.raw_file_dir)\
            .withColumn('week', col('week').cast('int'))\
            .withColumn('year', col('year').cast('int'))\
            .withColumn('month', col('month').cast('int'))\
            .withColumn('day_of_week', col('dayofweek').cast('int'))\
            .withColumn('day_of_month', col('dayofmonth').cast('int'))\
            .withColumn('day_of_year', col('dayofyear').cast('int'))\
            .select('date','week','year','month','day_of_week','day_of_month','day_of_year','week_part')
        df.write.mode("overwrite").saveAsTable(f"{self.catalog}.{self.schema}.date_lookup")
    
    def launch_history(self):
        print('Inserting history records...', end='')
        self.history_load()
        print('Done.')

In [0]:
class HistoryTestSuit():

    def __init__(self):
        obj_conf = ConfigModule()
        self.raw_file_dir = f"{obj_conf.base_file_path}/raw"
        self.catalog = obj_conf.environment
        self.schema = obj_conf.db_name
    
    def testcases(self, expected_count):
        print("Testing History tables...", end='')
        actual_count = spark.sql(f"select count(*) from {self.catalog}.{self.schema}.date_lookup").collect()[0][0]
        assert actual_count==expected_count, f"Test Failed, actual count is {actual_count}"
        print("Test Passed")

#obj = HistoryTestSuit()
#obj.testcases(expected_count=365)